In [1]:
#########################################################################
# LIBRERIAS NECESARIAS
#########################################################################
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import LoggerWrapper
import torch

import tensorflow as tf
from stable_baselines3 import PPO
from tensorflow.keras.layers import concatenate
import keras
import matplotlib.pyplot as plt
from keras import layers
import copy
from sklearn.model_selection import train_test_split
import csv

import os
from stable_baselines3.common.vec_env import DummyVecEnv

2023-07-08 11:46:46.748955: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-08 11:46:46.815777: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 11:46:47.737232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
############################################################################
# Variables globales
###########################################################################
BATCH_SIZE = 28032
EPOCHS =100
EPISODES = 1

In [3]:
# Definimos el entorno
env = gym.make('Eplus-5Zone-hot-discrete-v1')
env = LoggerWrapper(env)

# Obtenemos el espacio de estados del entorno
ob_space = env.observation_space
ac_space = env.action_space


[2023-07-08 11:46:50,048] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-07-08 11:46:50,048] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-07-08 11:46:50,050] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-07-08 11:46:50,051] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-07-08 11:46:50,052] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...


# Definición del modelo PPO y entrenamiento

In [4]:
model = PPO(policy="MlpPolicy", env=env, verbose=0)

# Entrenamiento

In [5]:
model.learn(total_timesteps=25)

[2023-07-08 11:46:50,074] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-08 11:46:50,216] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res21/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


# Evaluación del modelo PPO

In [6]:
reward = 0
success_num = 0

# Por cada episodio
for iteration in range(EPISODES):
    # Inicializo todas las variables
    observations = []
    actions = []
    rewards = []
    run_policy_steps = 0

    truncated = False
    terminated = False

    # La primera acción de cada episodio se crea con la red neuronal

    obs, _ = env.reset()


    # --Actualización de variables: ojo no introduzco el estado y accion inicial, solo introduzco los de PPO
    observations.append(obs)  # S_0

   
    # Por cada steps en cada episodio, mientras no se llegue a un
    # estado terminal o un estado malo
    while terminated != True and truncated != True:
        # --Aumentar el numero de steps
        run_policy_steps += 1

        action, _ = model.predict(obs)

        action = int(action)

        # --Muevo al Agente al siguiente estado
        next_obs, reward, terminated, truncated, info = env.step(action)

        # --Actualización de variables
        actions.append(action)  # A_i-1
        rewards.append(reward)  # R_i-1

        # --Si llegamos a un estado final, el juego ha finalizado!!!
        # --Se configura el tablero de nuevo
        if terminated:
            obs = env.reset()
            reward = -1
            break
        else:
            observations.append(next_obs)  # O_i
            obs = next_obs

    # Ver si el episodio ha obtendo una recompensa total igual o
    # superior a 195
    if sum(rewards) >= 195:
        success_num += 1
        if success_num >= 100:
            break
    else:
        success_num = 0

observations = np.reshape(observations,newshape=[-1] + list(ob_space.shape))

actions = np.array(actions).astype(dtype=np.int32)


[2023-07-08 11:46:58,587] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-08 11:46:58,588] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-08 11:46:58,679] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res21/Eplus-env-sub_run2
[2023-07-08 11:47:31,569] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-08 11:47:31,571] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-08 11:47:31,671] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res21/Eplus-env-sub_run3


# Evaluación

**A continuación, probamos como actuaría una agente con la política obtenida con PPO**

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(a

[2023-07-08 11:47:36,650] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-08 11:47:36,651] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-08 11:47:36,830] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res21/Eplus-env-sub_run4
[2023-07-08 11:48:12,865] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-07-08 11:48:12,867] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-07-08 11:48:12,980] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-5Zone-hot-discrete-v1-res21/Eplus-env-sub_run5
[2023-07-08 11:48:48,191] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully

In [8]:

print(f"Recompensa promedio: {mean_reward} +/- {std_reward}")


Recompensa promedio: -43938.541059728246 +/- 0.0
